# Notebook for Work with Optimization and Pool and Executor instances
For possible application in ModelFitting in brian2tools

Notebook goals: 
    - create instances with Scipy and Nevergrad that take two params and return optimal error
    - modify them to use the workers with Executor and Pool 
    - try using NeuronGrouop to adapt them

In [1]:
import numpy as np
import multiprocessing as mp

from scipy.optimize import differential_evolution, rosen
from scipy.optimize._differentialevolution import DifferentialEvolutionSolver

from nevergrad.optimization import optimizerlib
from concurrent import futures
from nevergrad import instrumentation as inst

## Scipy

In [2]:
def test(x):
    arg2 = x[0] - x[1]
    return arg2

In [43]:
bounds = [(-5, 5), (-10, 0)]
result = differential_evolution(test, bounds, maxiter=100)
result.x, result.fun

(array([-5.,  0.]), -5.0)

### use workers to map

In [4]:
with mp.Pool(2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [5]:
print(solver.x)
print(test(solver.x))

[-5.  0.]
-5.0


In [6]:
with DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=mp.Pool(2).map) as solver:
    solver.solve()

In [46]:
differential_evolution(test, bounds, updating='deferred', workers=mp.Pool(2).map, maxiter=10, disp=True)

differential_evolution step 1: f(x)= -3.7502
differential_evolution step 2: f(x)= -4.36675
differential_evolution step 3: f(x)= -4.36675
differential_evolution step 4: f(x)= -4.6628
differential_evolution step 5: f(x)= -4.6628
differential_evolution step 6: f(x)= -4.92209
differential_evolution step 7: f(x)= -4.92209
differential_evolution step 8: f(x)= -4.95139
differential_evolution step 9: f(x)= -4.95139
differential_evolution step 10: f(x)= -4.95139


     fun: -5.0
     jac: array([ 0.99999999, -0.99999999])
 message: 'Maximum number of iterations has been exceeded.'
    nfev: 336
     nit: 10
 success: False
       x: array([-5.,  0.])

In [8]:
print(solver.x)
print(test(solver.x))

[-5.  0.]
-5.0


### Using Like for The Nevergrad PoolExecutor with Scipy optimize

In [9]:
with futures.ProcessPoolExecutor(max_workers=2) as p, DifferentialEvolutionSolver(
    test, bounds, updating='deferred', workers=p.map) as solver:
    solver.solve()

In [10]:
solver.converged()

True

In [11]:
print(solver.x)
print(test(solver.x))

[-5.  0.]
-5.0


### Nevergrad

In [12]:
# define the variables to be a bounded array
arg1 = inst.var.Array(2).bounded(a_min=-5, a_max=5)

instrum = inst.Instrumentation(arg1)
instrum

Instrumentation(Array(shape=(2,), transforms=[ArctanBound(a_max=5, a_min=-5)]))

In [13]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)

In [14]:
optim.optimize(test)

Candidate(args=(array([-4.99986066,  4.99966793]),), kwargs={}, data=[-22844.64986779   9585.49783353])

#### Verbosity
you can set up verbosity parameter to report on fitness of the function

In [15]:
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10, num_workers=2)

with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(test, executor=executor, batch_mode=True, verbosity=2)

Launching 2 jobs with new suggestions


/home/alteska/anaconda3/envs/brian_opt/lib/python3.6/site-packages/nevergrad/optimization/differentialevolution.py:249: InefficientSettingsWarning: DE algorithms are inefficient with budget < 60
  warnings.warn("DE algorithms are inefficient with budget < 60", base.InefficientSettingsWarning)


Updating fitness with value 5.6830861428234245
8 remaining budget and 1 running jobs
Current pessimistic best is: Point<x: [ 1.41389318 -1.09383778], mean: 5.6830861428234245, count: 1>
Updating fitness with value -1.8751499427721732
8 remaining budget and 0 running jobs
Current pessimistic best is: Point<x: [-0.90366235 -0.14677976], mean: -1.8751499427721732, count: 1>
Launching 2 jobs with new suggestions
Updating fitness with value 0.007521666452447617
6 remaining budget and 1 running jobs
Current pessimistic best is: Point<x: [-0.90366235 -0.14677976], mean: -1.8751499427721732, count: 1>
Updating fitness with value -4.597210301692739
6 remaining budget and 0 running jobs
Current pessimistic best is: Point<x: [-2.35580618  0.28203274], mean: -4.597210301692739, count: 1>
Launching 2 jobs with new suggestions
Updating fitness with value 1.9155529087424477
4 remaining budget and 1 running jobs
Current pessimistic best is: Point<x: [-2.35580618  0.28203274], mean: -4.597210301692739,

/home/alteska/anaconda3/envs/brian_opt/lib/python3.6/site-packages/nevergrad/common/tools.py:168: UserWarning: Ignoring since timer was stopped before starting.
  warnings.warn("Ignoring since timer was stopped before starting.")


In [16]:
recommendation

Candidate(args=(array([-2.50346175,  2.51269829]),), kwargs={}, data=[-1.00217745  1.00801057])

### Nevergrad with Executor

In [17]:
# can not be used the other way around, needs "submit" method

# optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000, num_workers=2)
# with mp.Pool(processes=optim.num_workers) as executor:
#     recommendation = optim.optimize(test, executor=executor, batch_mode=True)

### Two Separate Bounds Problem
Multiple bounds can not be passed for an array, but array can be cast as scalar and then used. 

In [18]:
# This kind of approaches won't work:
# a_max = {5, 5}
# a_min = {-5, -5}
# arg1 = inst.var.Array(2).bounded(a_min=a_min, a_max=a_max)

# a_max = [5, 5]
# a_min = [-5, -5]
# arg1 = inst.var.Array(2).bounded(a_min=a_min, a_max=a_max)

#### Two Variables Approach

In [19]:
def test2(x, y):
    arg2 = x - y
    return arg2

In [20]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [21]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=5000)
optim.optimize(test2)

Candidate(args=(-4.999999998531852, 9.99999999891218), kwargs={}, data=[-2.16810411e+09  2.92612227e+09])

#### Args Approach

In [22]:
def test_args(*args):
    return args[0] - args[1]

In [23]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [24]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=1000)
with futures.ProcessPoolExecutor(max_workers=3) as executor:
    recommendation = optim.optimize(test_args, executor=executor, batch_mode=True, verbosity=1)

Launching 1 jobs with new suggestions
Updating fitness with value 0.3292055144540109
999 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
Updating fitness with value -4.772791228125528
998 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
Updating fitness with value -9.96628118025604
997 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
Updating fitness with value -2.1485594648894466
996 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
Updating fitness with value -5.488641915162891
995 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
Updating fitness with value -6.877154750857039
994 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
Updating fitness with value -6.608169355719731
993 remaining budget and 0 running jobs
Launching 1 jobs with new suggestions
Updating fitness with value -0.8835050737609764
992 remaining budget and 0 running jobs

In [25]:
recommendation

Candidate(args=(-4.999942785946464, 9.999912459186417), kwargs={}, data=[-55634.912487    36361.31229233])

### Test with More complicated function

In [26]:
def ackley(x):
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e

In [27]:
def ackley_args(*args):
    arg1 = -0.2 * np.sqrt(0.5 * (args[0] ** 2 + args[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * args[0]) + np.cos(2. * np.pi * args[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e

#### Scipy Verison of this optimization

In [28]:
bounds = [(-5, 5), (-10, 0)]

In [29]:
res = differential_evolution(ackley, bounds, updating='deferred', workers=mp.Pool(2).map, polish=False)
res

     fun: 4.440892098500626e-16
 message: 'Optimization terminated successfully.'
    nfev: 3990
     nit: 132
 success: True
       x: array([0., 0.])

#### Nevergrad

In [30]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=15000, num_workers=2)

In [31]:
with futures.ProcessPoolExecutor(max_workers=optim.num_workers) as executor:
    recommendation = optim.optimize(ackley_args, executor=executor, batch_mode=True, verbosity=0)

In [32]:
recommendation

Candidate(args=(1.1696421083317498e-11, 9.407434831132377e-10), kwargs={}, data=[ 3.67453905e-12 -3.38359821e+09])

## Using Instrumentation SoftmaxCategorical ( utils.Variable[X] ):
There are other ways to create variables, however they are not boundable

Discrete set of n values transformed to a n-dim continuous variable. Each of the dimension encodes a weight for a value, and the softmax of weights provide probabilities for each possible value. A random value is sampled from this distribution. Since the chosen value is drawn randomly, the use of this variable makes deterministic functions become stochastic, hence "adding noise"


Input: list
    a list of possible values for the variable

In [33]:
# in case of Array we can bound it's general output in following way
var = inst.var.Array(2, 2).bounded(3, 5, transform="arctan")
data = np.array([-10, 10, 0, 0])
output = var.data_to_argument(data)
output

array([[3.06345103, 4.93654897],
       [4.        , 4.        ]])

In [34]:
token = inst.var.SoftmaxCategorical(["blu", "blublu", "blublublu"], deterministic=True)
token.data_to_argument([1, 1, 1.01], deterministic=False)

'blublublu'

In [35]:
token = inst.var.SoftmaxCategorical(["blu", "blublu", "blublublu"])
token.data_to_argument([.5, 1, 2.])
token.data_to_argument(token.argument_to_data("blu"), deterministic=True)

'blu'

### Test Ask and Tell 

In [36]:
arg1 = inst.var.Array(1).bounded(-5, 5).asscalar()
arg2 = inst.var.Array(1).bounded(0, 10).asscalar()
instrum = inst.Instrumentation(arg1, arg2)

In [37]:
# pick the optimization method; budget = number of allowed evaluations
optim = optimizerlib.registry['DE'](instrumentation=instrum, budget=10000)

In [38]:
for _ in range(10):
    x = optim.ask()
    value = test_args(*x.args, **x.kwargs)
    optim.tell(x, value)

In [39]:
recommendation = optim.provide_recommendation()
recommendation

Candidate(args=(-3.6464679432764595, 6.164382934941992), kwargs={}, data=[-2.20821805  0.38304109])

In [40]:
optim.optimize(test_args)

Candidate(args=(-4.999843814445688, 9.99994140212845), kwargs={}, data=[-20380.23857768  54321.06623147])

In [41]:
recommendation.args

(-3.6464679432764595, 6.164382934941992)

In [42]:
recommendation.uuid

'dafb32957b9b4dfdb492f160257396ee'